## Reference on test images based on SSD_MobileNetV2 

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Những thư viện cần tải nếu chưa có

In [ ]:
#!pip install tensorflow==2.11.0


In [ ]:
#!pip install dash==1.20.0 

In [ ]:
#!pip install opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive
!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research


In [31]:
!pip install object_detection
!pip install tensorflow-object-detection-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import tensorflow.compat.v1 as tf
print(tf.__version__)
import tensorflow as tf 
import os
os.environ['PYTHONPATH'] += '/content/models/research/slim'


2.11.0


In [ ]:
#!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install --upgrade tf_slim
import tf_slim as slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
%cd /content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/model/real_model
lst = os.listdir('fine_tuned_model_real')
lf = filter(lambda k: 'model.ckpt' in k, lst)
last_model = sorted(lf)[0].replace('.meta', '')


!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/model/config/ssd_mobilenetV2_finetune_traffic_light_simu.config \
    --output_directory=real_model/fine_tuned_model_real \
    --trained_checkpoint_prefix=real_model/$last_model


/content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/model/real_model
2023-02-15 06:29:14.519351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-15 06:29:14.519514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-15 06:29:14.519540: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent c

## frozen the model
đào tạo khoảng 15000 lần lặp

In [35]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor

from glob import glob
import os 
os.chdir('/content/models/research/object_detection')
import sys
sys.path.append('/content/models/research/object_detection')


from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [36]:
# Đường dẫn đến file graph: udacity_traffic_light_detection/google_colab_work/work/model/real_model/fine_tuned_model_real/frozen_inference_graph.pb
SSD_GRAPH_FILE = '/content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/model/real_model/fine_tuned_model_real/frozen_inference_graph.pb'
#Đường dẫn đến file gắn nhãn: udacity_traffic_light_detection/google_colab_work/work/model/real_model/fine_tuned_model_real/labels_map.pbtxt
PATH_TO_LABELS = '/content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/model/real_model/fine_tuned_model_real/labels_map.pbtxt'
#số lớp trên mô hình đã đào tạo và file gắn nhãn
NUM_CLASSES = 4


In [37]:
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef() 
        with tf.compat.v1.io.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [38]:
detection_graph = load_graph(SSD_GRAPH_FILE)

In [39]:
#Chỉnh sửa lại thư viện label_map_util.py nếu lỗi: thay thế dòng 132 thành with tf.io.gfile.GFile(path, 'r') as fid: và khởi động lại thời gian chạy
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'Green'}, 2: {'id': 2, 'name': 'Yellow'}, 3: {'id': 3, 'name': 'Red'}, 4: {'id': 4, 'name': 'off'}}


num_detections can be changed in the configure file, default is 100 

In [40]:
PATH_TO_IMGS = '/content/drive/MyDrive/DO_AN_NGANH/Phat_hien_phan_loai_den_giao_thong/traffic_light_detection_dataset/real_test_imgs'
TEST_IMGS = glob(os.path.join(PATH_TO_IMGS, '*.jpg'))
IMAGE_SIZE = (20, 15)
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
  
      for idx, img_path in enumerate(TEST_IMGS):
          image = Image.open(img_path)
          image_np = load_image_into_numpy_array(image)
          image_expanded = np.expand_dims(image_np, axis=0)
            
          (boxes, scores, classes, num) = sess.run(
              [detect_boxes, detect_scores, detect_classes, num_detections],
              feed_dict={image_tensor: image_expanded})
            
          print('SCORES')
          print(scores.shape)
          print('CLASSES')
          print(classes.shape)
            
          vis_util.visualize_boxes_and_labels_on_image_array(
              image_np, 
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              max_boxes_to_draw=5,
              line_thickness=4)
          plt.figure(figsize=IMAGE_SIZE)
          plt.imshow(image_np)
          plt.show()
            
          if idx == 10:
              break

Output hidden; open in https://colab.research.google.com to view.